# Лабораторная работа №4 Проведение исследований со случайным лесом

Подключим необходимые базовые модули

In [ ]:
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, root_mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer

In [ ]:
kagglehub.dataset_download("yasserh/wine-quality-dataset")
kagglehub.dataset_download("shree1992/housedata")

100%|██████████| 21.5k/21.5k [00:00<00:00, 13.9MB/s]

Extracting files...


100%|██████████| 432k/432k [00:00<00:00, 69.7MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2'

Чтобы уменьшить повторение кода напишем функции для подготовки базовых датасетов

In [ ]:
def prepare_train_test_split_class(dataset_path, test_size=0.2, random_state=7575):
    df = pd.read_csv(dataset_path)

    if 'Id' in df.columns:
        df = df.drop(columns=['Id'])

    y = df['quality']
    X = df.drop(columns=['quality'])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=True
    )

    return X_train, X_test, y_train, y_test

In [ ]:
def prepare_train_test_split_reg(dataset_path, test_size=0.2, random_state=7575):
    df = pd.read_csv(dataset_path)

    drop_columns = ['street', 'country', 'date']
    df = df.drop(columns=[col for col in drop_columns if col in df.columns])

    categorical_features = ['city', 'statezip']
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    encoder.fit(df[categorical_features])
    encoded_cols = encoder.get_feature_names_out(categorical_features)
    encoded_df = pd.DataFrame(encoder.transform(df[categorical_features]), columns=encoded_cols, index=df.index)
    df = pd.concat([df.drop(columns=categorical_features), encoded_df], axis=1)

    y = df['price']
    X = df.drop(columns=['price'])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=True
    )

    return X_train, X_test, y_train, y_test


Загрузим датасеты для обучения базовых моделей

In [ ]:
X_train_class, X_test_class, y_train_class, y_test_class = prepare_train_test_split_class(
    "/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv",
    0.2,
    65366
)

In [ ]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = prepare_train_test_split_reg(
    "/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2/data.csv",
    0.2,
    653634
)

## 2. Создание бейзлайна и оценка качества

Для начала подключим и обучим модель из sklearn

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [ ]:
forest_class = RandomForestClassifier(random_state=758659)
forest_class.fit(X_train_class, y_train_class)

RandomForestClassifier(random_state=758659)

In [ ]:
forest_reg = RandomForestRegressor(random_state=4248)
forest_reg.fit(X_train_reg, y_train_reg)

RandomForestRegressor(random_state=4248)

Оценим результаты работы датасета

In [ ]:
y_pred_class = forest_class.predict(X_test_class)
y_pred_reg = forest_reg.predict(X_test_reg)

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

rmse_reg = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print("Classification - Random Forest:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

print("Regression - Random Forest:")
print(f"RMSE: {rmse_reg:.4f}")
print(f"R^2 Score: {r2_reg:.4f}")

Classification - Random Forest:
Accuracy: 0.6900
F1 Score: 0.6801

Regression - Random Forest:
RMSE: 216876.3926
R^2 Score: 0.6460


Можем заметить что рандомый лес на сырых данных показало чуть ли нелучшие результаты среди всех моделей. Возможно на улучшенном бейзлайне результаты станут еще выше. Возможно столько высокие результаты связаны с использованием ансабля моделей, которые лежат в основе случайного леса

## 3. Улучшение бейзлайна и оценка качества

В данной лабораторной работе я решил убрал oversampling в данной задаче он не дал нужного эффекте при обучении
Для регрессии я решил отказался от логарифмирования целевой переменной, т.к. это может сильно влиять на RMSE

Также в задаче классификации я решил оставить создание нового признака

Итоговые гипотезы для данной лабораторной работы
- Классификация
  1. Убрать признаки residual sugar, pH
  2. Прологарифмируем признаки chlorides и sulphates
  3. Создадим признак-отношение Free Sulfur Dioxide и Total Sulfur Dioxide
- Регрессия
  1. Удалить признак sqft_lot, condition, yr_build, yr_renovated из-за низкой корреляции
  2. Попробуем перевести признак waterfront в бинарный
  3. Обрежем выбросы для bedrooms, bathrooms и price, sqft_living, sqft_above, sqft_basement

### Обучим новые модели на новом бейзлайне

In [ ]:
df_class = pd.read_csv("/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv")
df_class = df_class.drop(columns=['Id'])

df_class = df_class.drop(columns=['residual sugar', 'pH'])

df_class['chlorides'] = np.log1p(df_class['chlorides'])
df_class['sulphates'] = np.log1p(df_class['sulphates'])

df_class['sulfur_ratio'] = df_class['free sulfur dioxide'] / df_class['total sulfur dioxide']

y_class = df_class['quality']
X_class = df_class.drop(columns=['quality'])

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=5435, shuffle=True
)

In [ ]:
df_reg = pd.read_csv("/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2/data.csv")
df_reg = df_reg.drop(columns=['yr_built', 'sqft_lot', 'condition', 'yr_renovated', 'country', 'street', 'date'])

categorical_features = ['city', 'statezip']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_array = encoder.fit_transform(df_reg[categorical_features])
encoded_cols = encoder.get_feature_names_out(categorical_features)

encoded_df = pd.DataFrame(encoded_array, columns=encoded_cols, index=df_reg.index)

df_reg = pd.concat([df_reg.drop(columns=categorical_features), encoded_df], axis=1)

numerical_features = ['price', 'sqft_living', 'sqft_above', 'sqft_basement', 'bathrooms', 'bedrooms']

for col in numerical_features:
    Q1 = df_reg[col].quantile(0.25)
    Q3 = df_reg[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_reg = df_reg[(df_reg[col] >= lower_bound) & (df_reg[col] <= upper_bound)]

y_reg = df_reg['price']
X_reg = df_reg.drop(columns=['price'])

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=54356, shuffle=True
)


Обучим наши модели

In [ ]:
forest_class_improve = RandomForestClassifier(random_state=85877)
forest_class_improve.fit(X_train_class, y_train_class)

RandomForestClassifier(random_state=85877)

In [ ]:
forest_reg_improve = RandomForestRegressor(random_state=957)
forest_reg_improve.fit(X_train_reg, y_train_reg)

RandomForestRegressor(random_state=957)

Оценим получившиеся результаты

In [ ]:
y_pred_class = forest_class_improve.predict(X_test_class)
y_pred_reg = forest_reg_improve.predict(X_test_reg)

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

rmse_reg = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print("Classification - Random Forest Improve:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

print("Regression - Random Forest Improve:")
print(f"RMSE: {rmse_reg:.4f}")
print(f"R^2 Score: {r2_reg:.4f}")

Classification - Random Forest Improve:
Accuracy: 0.6900
F1 Score: 0.6776

Regression - Random Forest Improve:
RMSE: 112026.7334
R^2 Score: 0.7198


### Сравнение бейзлайнов

- Классификация

|          | Первый baseline | Улучшенный baseline |
|----------|-----------------|---------------------|
| Accuracy | 0.6900          | 0.6900              |
| F1       | 0.6801          | 0.6776              |

- Регрессия

|       | Первый baseline | Улучшенный baseline |
|-------|-----------------|---------------------|
| RMSE  | 216876.39     | 112026.73           |
| $R^2$ | 0.6460          | 0.7198              |

Как можем заметить в задаче классификации новый бейзлайн не сильно повлиял на метрики. Модель уже на сырых данных показала достаточно хорошие результаты. Случайны лес всегда хорошо показывает себя в задаче классификации.
В регрессии же получилось еще сильне улучшить показатель $R^2$

## 4. Имплементация алгоритма машинного обучения

Теперь реализуем свои версии случайного леса. Пойдем тем же путем, что и в прошлых лабораторных работах, реализуем универсальный класс, который будем параметризовать в зависимости от задачи

In [ ]:
from sklearn.utils import resample

# Используем решающее дерево из прошлой лабораторной работы
class DecisionTreeModel:
    def __init__(self, mode="classification", max_levels=None):
        self.mode = mode
        self.max_levels = max_levels
        self.root = None

    def train(self, features, labels):
        self.root = self._grow_tree(features, labels, current_depth=0)

    def predict(self, features):
        return np.array([self._navigate_tree(sample, self.root) for sample in features])

    def _grow_tree(self, features, labels, current_depth):
        if self.max_levels is not None and current_depth >= self.max_levels:
            return self._create_terminal_node(labels)

        if len(set(labels)) == 1:
            return self._create_terminal_node(labels)

        split_feature, split_value = self._identify_optimal_split(features, labels)
        if split_feature is None:
            return self._create_terminal_node(labels)

        left_indices = features[:, split_feature] <= split_value
        right_indices = features[:, split_feature] > split_value

        left_branch = self._grow_tree(features[left_indices], labels[left_indices], current_depth + 1)
        right_branch = self._grow_tree(features[right_indices], labels[right_indices], current_depth + 1)

        return {
            "split_feature": split_feature,
            "split_value": split_value,
            "left": left_branch,
            "right": right_branch
        }

    def _identify_optimal_split(self, features, labels):
        optimal_feature, optimal_value = None, None
        best_metric = float("inf") if self.mode == "regression" else -float("inf")

        for feature_idx in range(features.shape[1]):
            unique_values = np.unique(features[:, feature_idx])
            for value in unique_values:
                left_mask = features[:, feature_idx] <= value
                right_mask = features[:, feature_idx] > value

                if sum(left_mask) == 0 or sum(right_mask) == 0:
                    continue

                metric = self._evaluate_split(labels, left_mask, right_mask)

                if (self.mode == "regression" and metric < best_metric) or (
                    self.mode == "classification" and metric > best_metric
                ):
                    best_metric = metric
                    optimal_feature = feature_idx
                    optimal_value = value

        return optimal_feature, optimal_value

    def _evaluate_split(self, labels, left_mask, right_mask):
        if self.mode == "regression":
            left_error = np.var(labels[left_mask]) * sum(left_mask)
            right_error = np.var(labels[right_mask]) * sum(right_mask)
            return left_error + right_error
        else:
            left_score = self._compute_gini(labels[left_mask]) * sum(left_mask)
            right_score = self._compute_gini(labels[right_mask]) * sum(right_mask)
            return -(left_score + right_score)

    def _compute_gini(self, labels):
        proportions = np.bincount(labels) / len(labels)
        return 1 - np.sum(proportions ** 2)

    def _create_terminal_node(self, labels):
        if self.mode == "regression":
            return np.mean(labels)
        else:
            return np.bincount(labels).argmax()

    def _navigate_tree(self, sample, node):
        if not isinstance(node, dict):
            return node

        if sample[node["split_feature"]] <= node["split_value"]:
            return self._navigate_tree(sample, node["left"])
        else:
            return self._navigate_tree(sample, node["right"])

class RandomForest:
    def __init__(self, n_estimators=100, max_features='sqrt', max_depth=None, task='classification'):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.task = task
        self.trees = []

        if task == 'classification':
            self.mode = 'classification'
        elif task == 'regression':
            self.mode = 'regression'
        else:
            raise ValueError("Task must be 'classification' or 'regression'")

    def fit(self, X, y):
        n_samples, n_features = X.shape
        for _ in range(self.n_estimators):
            X_sample, y_sample = resample(X, y, n_samples=n_samples, replace=True)
            tree = DecisionTreeModel(mode=self.mode, max_levels=self.max_depth)
            tree.train(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])

        if self.task == 'classification':
            majority_votes = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)
            return majority_votes
        elif self.task == 'regression':
            mean_prediction = np.mean(predictions, axis=0)
            return mean_prediction

Подготовим базовые датасеты и запустим обучение имплементированных моделей

In [ ]:
X_train_class, X_test_class, y_train_class, y_test_class = prepare_train_test_split_class(
    "/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv",
    0.2,
    65366
)

In [ ]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = prepare_train_test_split_reg(
    "/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2/data.csv",
    0.2,
    653634
)

In [ ]:
forest_class_impl = RandomForest(max_depth=5, task="classification")
forest_class_impl.fit(X_train_class.to_numpy(), y_train_class.to_numpy())

In [ ]:
forest_reg_impl = RandomForest(max_depth=5, n_estimators=20, task='regression')
forest_reg_impl.fit(X_train_reg.to_numpy(), y_train_reg.to_numpy())

Оценим полученные модели

In [ ]:
y_pred_class = forest_class_impl.predict(X_test_class.to_numpy())
y_pred_reg = forest_reg_impl.predict(X_test_reg.to_numpy())

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

rmse_reg = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print("Classification - Random Forest Impl:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

print("Regression - Random Forest Impl:")
print(f"RMSE: {rmse_reg:.4f}")
print(f"R^2 Score: {r2_reg:.4f}")

Classification - Random Forest Impl:
Accuracy: 0.6157
F1 Score: 0.6018

Regression - Random Forest Impl:
RMSE: 262082.2301
R^2 Score: 0.4830


### Сравнение с реализацией в библиотеке

- Классификация

|          | Библиотека      | Имплементация       |
|----------|-----------------|---------------------|
| Accuracy | 0.6900          | 0.6157              |
| F1       | 0.6801          | 0.6018              |

- Регрессия

|        | Библиотека      | Имплиментация    |
|--------|-----------------|------------------|
| RMSE   | 216876.39       | 262082.23        |
| $R^2$  | 0.6460          | 0.4830           |

Как можем заметить собственная реализация в этот раз сильно уступает библиотечной. Не удивительно, т.к. существует большое количество методов для организации ансабля решающих деревьев. Также моя имплементация долго работала при установлении количества деревьев в 100, в результате чего пришлось сократить их количество

### Обучим модель на улучшенном бейзлайне

Подготовим улучшенный бейзлайн

In [ ]:
df_class = pd.read_csv("/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv")
df_class = df_class.drop(columns=['Id'])

df_class = df_class.drop(columns=['residual sugar', 'pH'])

df_class['chlorides'] = np.log1p(df_class['chlorides'])
df_class['sulphates'] = np.log1p(df_class['sulphates'])

df_class['sulfur_ratio'] = df_class['free sulfur dioxide'] / df_class['total sulfur dioxide']

y_class = df_class['quality']
X_class = df_class.drop(columns=['quality'])

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=5435, shuffle=True
)

In [ ]:
df_reg = pd.read_csv("/root/.cache/kagglehub/datasets/shree1992/housedata/versions/2/data.csv")
df_reg = df_reg.drop(columns=['yr_built', 'sqft_lot', 'condition', 'yr_renovated', 'country', 'street', 'date'])

categorical_features = ['city', 'statezip']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_array = encoder.fit_transform(df_reg[categorical_features])
encoded_cols = encoder.get_feature_names_out(categorical_features)

encoded_df = pd.DataFrame(encoded_array, columns=encoded_cols, index=df_reg.index)

df_reg = pd.concat([df_reg.drop(columns=categorical_features), encoded_df], axis=1)

numerical_features = ['price', 'sqft_living', 'sqft_above', 'sqft_basement', 'bathrooms', 'bedrooms']

for col in numerical_features:
    Q1 = df_reg[col].quantile(0.25)
    Q3 = df_reg[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_reg = df_reg[(df_reg[col] >= lower_bound) & (df_reg[col] <= upper_bound)]

y_reg = df_reg['price']
X_reg = df_reg.drop(columns=['price'])

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=54356, shuffle=True
)


Обучим модели на улучшенном бейзлайне

In [ ]:
forest_class_impl_improve = RandomForest(max_depth=5, task="classification")
forest_class_impl_improve.fit(X_train_class.to_numpy(), y_train_class.to_numpy())

In [ ]:
forest_reg_impl_improve = RandomForest(max_depth=5, n_estimators=20, task='regression')
forest_reg_impl_improve.fit(X_train_reg.to_numpy(), y_train_reg.to_numpy())

Оценим результаты получившихся моделей

In [ ]:
y_pred_class = forest_class_impl_improve.predict(X_test_class.to_numpy())
y_pred_reg = forest_reg_impl_improve.predict(X_test_reg.to_numpy())

accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

rmse_reg = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print("Classification - Random Forest Impl:")
print(f"Accuracy: {accuracy_class:.4f}")
print(f"F1 Score: {f1_class:.4f}\n")

print("Regression - Random Forest Impl:")
print(f"RMSE: {rmse_reg:.4f}")
print(f"R^2 Score: {r2_reg:.4f}")

Classification - Random Forest Impl:
Accuracy: 0.6201
F1 Score: 0.5991

Regression - Random Forest Impl:
RMSE: 139889.7136
R^2 Score: 0.5631


Результаты получились немного противоречивыми, с одной стороны, такие параметры как Accuracy и RMSE улучшились, с другой стороны F1 и $R^2$ ухудшились. Возможно это заключается в особенностях имлементации алгоритмов

### Сравним результаты библиотеки и имлементации на улучшенном бейзлайне

- Классификация

|          | Библиотека      | Имплементация       |
|----------|-----------------|---------------------|
| Accuracy | 0.6900          | 0.6201              |
| F1       | 0.6776          | 0.5991              |

- Регрессия

|        | Библиотека      | Имплиментация    |
|--------|-----------------|------------------|
| RMSE   | 112026.73       | 139889.71        |
| $R^2$  | 0.7198          | 0.5631           |

Можно заметить, что в данной лабораторной работе реализованный алгоритм и библиотечная функция сильно отличаются по точности. Скорее всего это заключается в особенностях реализации алгоритма и насторойке различных параметров

В целом рандомный лес показал себя очень мощный алгоритмом, который показал одни из лучших результатов среди всех моделей. Возможно с большим анализом и перебором гиперпараметров получится еще улучшить модель